In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, applications, callbacks
import matplotlib.pyplot as plt
import numpy as np

In [4]:

TRAIN_PATH = "Plant Counting/train/train.tfrecord"
VALID_PATH = "Plant Counting/valid/valid.tfrecord"
TEST_PATH = "Plant Counting/test/test.tfrecord"

BATCH_SIZE = 32
EPOCHS = 50
IMG_SIZE = (224, 224)
AUTOTUNE = tf.data.AUTOTUNE

In [5]:
def decode_tfrecord(example):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'count': tf.io.FixedLenFeature([], tf.int64)
    }
    example = tf.io.parse_single_example(example, feature_description)
    image = tf.image.decode_jpeg(example['image'], channels=3)
    image = tf.image.resize(image, IMG_SIZE) / 255.0
    label = tf.cast(example['count'], tf.float32)
    return image, label

def load_dataset(path):
    dataset = tf.data.TFRecordDataset(path)
    dataset = dataset.map(decode_tfrecord, num_parallel_calls=AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    return dataset

In [6]:
train_ds = load_dataset(TRAIN_PATH)
valid_ds = load_dataset(VALID_PATH)
test_ds = load_dataset(TEST_PATH)

In [7]:
base_model = applications.MobileNetV2(input_shape=IMG_SIZE + (3,), include_top=False, weights='imagenet')
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1)
])


In [8]:

model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])

early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', patience=3)

history = model.fit(train_ds, validation_data=valid_ds, epochs=EPOCHS, callbacks=[early_stop, reduce_lr])


Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node ParseSingleExample/ParseExample/ParseExampleV2 defined at (most recent call last):
<stack traces unavailable>
Feature: count (data type: int64) is required but could not be found.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]]
	 [[IteratorGetNext]] [Op:__inference_one_step_on_iterator_8586]

In [ ]:
results = model.evaluate(test_ds)
y_true = []
y_pred = []



In [ ]:
for images, labels in test_ds:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(preds.flatten())

plt.scatter(y_true, y_pred)
plt.xlabel('Actual Count')
plt.ylabel('Predicted Count')
plt.title('Actual vs Predicted Plant Count')
plt.show()


In [ ]:
rmse = np.sqrt(np.mean((np.array(y_true) - np.array(y_pred)) ** 2))
r2 = 1 - (np.sum((np.array(y_true) - np.array(y_pred)) ** 2) / np.sum((np.array(y_true) - np.mean(y_true)) ** 2))

print("Test RMSE:", rmse)
print("Test R²:", r2)



In [ ]:

model.save('/kaggle/working/Plant_Counting_Upgraded_Model.keras')